In [1]:
from models.create_dataset import create_dataset
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
# lasso model
from epftoolbox.models import LEAR
from sklearn.linear_model import Lasso
import logging
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import hyperopt

In [10]:
# create dataset
target_col = 'DK1_price'
df = create_dataset(target_col)
y = df[target_col]
X = df.drop(target_col, axis=1)

['NO_Solar', 'NO_Solar_lag_1', 'NO_Solar_lag_2', 'NO_Solar_lag_3', 'NO_Solar_lag_7']


In [11]:
# Pivot hourly index out to columns so index is only date
pivot_columns = [col for col in X.columns if not col.startswith('day_of_week')]
X = X.pivot_table(index=X.index.date, columns=X.index.hour, values=X.columns)
X.columns = [f'hour_{col}' for col in X.columns]
# Drop Nan rows
X = X.dropna()

In [ ]:
# now y
y = pd.DataFrame({target_col: y.DK1_price}, index=y.index)
y = y.pivot_table(index=y.index.date, columns=y.index.hour, values=y.columns)
y.columns = [f'hour_{col}' for col in y.columns]
y = y.dropna()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, shuffle=False)


In [ ]:
class DNN4(nn.Module):
    def __init__(self, input_dim, output_dim, num_neurons, dropout_rate):
        # assure lenght of num_neurons is 3
        assert len(num_neurons) == 3

        super(DNN4, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.layers = layers
        self.fc1 = nn.Linear(self.input_dim, num_neurons[0])
        self.fc2 = nn.Linear(num_neurons[0], num_neurons[1])
        self.fc3 = nn.Linear(num_neurons[1], num_neurons[2])
        self.fc4 = nn.Linear(num_neurons[2], self.output_dim)
        self.dropout = nn.Dropout(0.2)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.relu(self.fc3(x))
        x = self.dropout(x)
        x = self.fc4(x)
        return x

In [ ]:
# make dataloader
# and small test to see if it works
train_loader = DataLoader(dataset=list(zip(X_train.values, y_train.values)), batch_size=32, shuffle=False)

In [ ]:
# Tree structured parzen estimator
from hyperopt import fmin, tpe, hp, Trials

criterion = nn.MSELoss() # maybe not this one, but for now
epochs = 20
# Define the search space
space = {
    'batch_size': hp.choice('batch_size', [32, 64, 128, 256]),
    'learning_rate': hp.loguniform('learning_rate', -5, -1),
    'num_neurons_1': hp.quniform('num_neurons', 32, 512, 32),
    'num_neurons_2': hp.quniform('num_neurons', 32, 512, 32),
    'num_neurons_3': hp.quniform('num_neurons', 32, 512, 32),
    'dropout_rate': hp.uniform('dropout_rate', 0.1, 0.5)
}

# Define the objective function
def objective(params, X_train=X_train, y_train=y_train, X_val=X_val, y_val=y_val, input_dim=X_train.shape[1], output_dim=24, epochs=epochs):
    # Train and evaluate your model with the given hyperparameters
    # Return the validation accuracy or other metric you want to optimize
    model = DNN4(input_dim, output_dim, [params['num_neurons_1'], params['num_neurons_2'], params['num_neurons_3']], params['dropout_rate'])
    optimizer = optim.Adam(model.parameters(), lr=params['learning_rate'])
    # __loader__
    train_loader = DataLoader(dataset=list(zip(X_train.values, y_train.values)), batch_size=params['batch_size'], shuffle=False)
    val_loader = DataLoader(dataset=list(zip(X_val.values, y_val.values)), batch_size=params['batch_size'], shuffle=False)
    train_losses = []
    val_losses = []
    for epoch in range(epochs):
        train_loss = 0.0
        val_loss = 0.0
        for i, (inputs, labels) in enumerate(train_loader):
            optimizer.zero_grad()
            outputs = model(inputs.float())
            loss = criterion(outputs, labels.float())
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        for i, (inputs, labels) in enumerate(val_loader):
            outputs = model(inputs.float())
            loss = criterion(outputs, labels.float())
            val_loss += loss.item()

        train_losses.append(train_loss/len(train_loader))
        val_losses.append(val_loss/len(val_loader))

    accuracy = val_losses[-1]


    return accuracy

# Define the TPE algorithm
tpe_algorithm = tpe.suggest

# Define the number of iterations
max_evals = 100

# Initialize the trials object
trials = Trials()

# Run the TPE algorithm to optimize the hyperparameters
best_params = fmin(objective, space, algo=tpe_algorithm, max_evals=max_evals, trials=trials)

# Print the best hyperparameters
print("Best hyperparameters:", best_params)


In [ ]:
# Linear NN
layers = 4
input_dim = X_train.shape[1]
output_dim = 24

